Download Flooding data

In [ ]:
from crawlerF.globalFloodDatabase import globalFloodDatabase
globalFloodDatabase().downloadAll("download")
# Finished on 2025/06/06

Download Population data

In [ ]:
from crawlerF.globalPopulation import globalPopulation
globalPopulation().downloadAll("test")
# Finished from 2025/06/07 to 2025/06/11
# IDN、IND、IRN、IRQ、ISR、JOR、JPN、KAZ、KGZ、KWT、LAO、LBN have problem, re-download from 2025/07/21 - 2025/07/29
import os
a = globalPopulation()
for country in ["IDN", "IND", "IRN", "IRQ", "ISR", "JOR", "JPN", "KAZ", "KGZ", "KWT", "LAO", "LBN"]:
    a.downloadOneCountryByISO(os.path.join("C:\\0_PolyU\\population_tmp", country), country)

Remove permanent water

In [ ]:
from raster.floodingMerge import floodingMerge
import os
"""
Useful band
Band 1: flooded
Band 2: flood_duration
Band 5: jrc_perm_water (1 - permanent water, 0 - non-water)
"""
flooding = floodingMerge(r"C:\\0_PolyU\\flooding")
# Flooding times
flooding.readAllTifInZip("C:\\0_PolyU\\floodingAll_Times", 1, multiThread=os.cpu_count()) # type: ignore
# Flooding days
flooding.readAllTifInZip("C:\\0_PolyU\\floodingAll_Days", 2, multiThread=os.cpu_count()) # type: ignore
# Flooding statistic period
flooding.calculateStasticPeriod("C:\\0_PolyU\\flooding")
# Finished on 2025/07/09

Merge population

In [ ]:
from raster.populationMerge import populationMerge
# All population
populationMerge(r"C:\\0_PolyU\\population\\", blockSize=2048).mergeAll(r"C:\\0_PolyU\\population_All", multiThread=4)
# All Male population
populationMerge(r"C:\\0_PolyU\\population\\", blockSize=2048).mergeAll(r"C:\\0_PolyU\\population_All", gender=['m'], multiThread=4)
# All Female population
populationMerge(r"C:\\0_PolyU\\population\\", blockSize=2048).mergeAll(r"C:\\0_PolyU\\population_Female", gender=['f'], multiThread=4)
# Different age group
ageGroup = {
        "children": [x for x in range(0, 25, 5)] + [1],
        "young": [x for x in range(25, 45, 5)],
        "middle": [x for x in range(45, 60, 5)],
        "elderly": [x for x in range(60, 81, 5)] # [x for x in range(60, 75, 5)]
        # "senile&long_living": [x for x in range(75, 81, 5)]
    }
for group in ["children", "young", "middle", "elderly"]:
    populationMerge(r"C:\\0_PolyU\\population\\", blockSize=4096) \
        .mergeAll("C:\\0_PolyU\\population_All_{}".format(group), mainAge=ageGroup[group], multiThread=4)

Get road network graph

In [ ]:
from nodeAnalysis.simpleRoad import getSimpleRoad
# Only obtain roads of level three or above
customFilter = " \
    [\"highway\"~\"^motorway','trunk','primary','secondary','tertiary','motorway_link$| \
    ^trunk_link','primary_link','secondary_link','tertiary_link$\"] \
"
getSimpleRoad().getAllCountriesNetworksGraph("C:\\0_PolyU\\roadsGraph", customFilter=customFilter, multiThread=os.cpu_count()) # type: ignore

# Countries that are too large to processed with osmnx origion method, which needs to be processed by offline osm.pbf file and transformed directly to graph:
## Sint Maarten
getSimpleRoad().getOneCountry("SINT MAARTEN", "C:\\0_PolyU\\roadsGraph", customFilter=customFilter)

customFilter = [
    "motorway", "trunk", "primary", "secondary", "tertiary", "motorway_link",
    "trunk_link", "primary_link", "secondary_link", "tertiary_link"
]
## Russia
getSimpleRoad().getOneCountryFromFile(("C:\\0_PolyU\\russia-latest.osm.pbf", "lines"), "RUS", "C:\\0_PolyU\\roadsGraph", customFilter=customFilter)
## Norway
getSimpleRoad().getOneCountryFromFile(("C:\\0_PolyU\\norway-latest.osm.pbf", "lines"), "NOR", "C:\\0_PolyU\\roadsGraph", customFilter=customFilter)
## New Caledonia
getSimpleRoad().getOneCountryFromFile(("C:\\0_PolyU\\new-caledonia-latest.osm.pbf", "lines"), "NCL", "C:\\0_PolyU\\roadsGraph", customFilter=customFilter)
## Kiribati
getSimpleRoad().getOneCountryFromFile(("C:\\0_PolyU\\kiribati-latest.osm.pbf", "lines"), "KIR", "C:\\0_PolyU\\roadsGraph", customFilter=customFilter)
## Pitcairn Islands
getSimpleRoad().getOneCountryFromFile(("C:\\0_PolyU\\pitcairn-islands-latest.osm.pbf", "lines"), "PCN", "C:\\0_PolyU\\roadsGraph", customFilter=customFilter + ["residential", "residential_link", "unclassified"])
## Tokelau
getSimpleRoad().getOneCountryFromFile(("C:\\0_PolyU\\tokelau-latest.osm.pbf", "lines"), "TKL", "C:\\0_PolyU\\roadsGraph", customFilter=customFilter + ["residential", "residential_link", "unclassified"])
## China
getSimpleRoad().getOneCountryFromFile(("D:\\origionalOSMFile\\china-lastest.gdb", "china_lastest"), "CHN", "C:\\0_PolyU\\roadsGraph", customFilter=customFilter)
## Check Results
getSimpleRoad().checkCountry("C:\\0_PolyU\\roadsGraph")

In [ ]:
import geopandas as gpd
place = gpd.read_file(r"_GISAnalysis\Dissertation.gdb", layer = "iso2_geom", encoding="utf-8")
border = place.loc[place["iso2"].isin(["C_", "_E", "_D"])]
border['geometry'] = border['geometry'].buffer(0)
border = border[border.is_valid & ~border.is_empty]
border_union = border.union_all()
border_union

In [23]:
a = gpd.GeoDataFrame({"geometry": [border_union]})
a = a.set_crs(4326)
a.to_file(r"test/a.geojson", encoding="utf-8")

In [ ]:
import geopandas as gpd
b=gpd.read_file(r"D:\origionalOSMFile\china-latest.osm.pbf", layer = "lines", encoding="utf-8")
b.to_file("D:\\origionalOSMFile\\china-lastest2.gpkg", encoding="utf-8")

In [3]:
customFilter = [
    "motorway", "trunk", "primary", "secondary", "tertiary", "motorway_link",
    "trunk_link", "primary_link", "secondary_link", "tertiary_link"
]
gdf = b[b["highway"].isin(customFilter)].copy()
gdf.to_file("D:\\origionalOSMFile\\china-lastest.gpkg", encoding="utf-8")